# MountainCar-v0 Demo

## This activity was done using the DQN method

- A car is on a one-dimensional track, positioned between two "mountains". 
- The goal is to drive up the mountain on the right; 
- however, the car's engine is not strong enough to scale the mountain in a single pass. 
- Therefore, the only way to succeed is to drive back and forth to build up momentum.

## Load Packages

In [4]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.optimizers import Adam
from keras import backend as K
import matplotlib.pyplot as plt
from IPython.display import HTML

## DQN Class

In [5]:
EPISODES = 400

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=4000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.997
        self.learning_rate = 0.002
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(64, input_dim=2))
        model.add(Activation('relu'))

        model.add(Dense(64))
        model.add(Activation('relu'))

        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss="mean_squared_error",
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def update_target_model(self):
        # copy weights from model to target_model
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = self.model.predict(state)
            if done:
                target[0][action] = reward
            else:
                Q_future  = self.target_model.predict(next_state)[0]
                target[0][action] = reward + self.gamma * np.amax(Q_future)
            self.model.fit(state, target, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save(self, name):
        self.model.save(name)

## Evaluation Algorithm (Main Method)

In [8]:
env = gym.make('MountainCar-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

print('state size:' ,state_size)
print('action size: ', action_size)

done = False
batch_size = 128

scores = 0
for e in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    flag = 0
    for time in range(10): #10
        action = agent.act(state)
        next_state, reward, done, info = env.step(action)

        if next_state[1] > state[0][1] and next_state[1]>0 and state[0][1]>0:
            reward += 15
        elif next_state[1] < state[0][1] and next_state[1]<=0 and state[0][1]<=0:
            reward -=15

        # give more reward if the cart reaches the flag in 200 steps
        if done:
            reward = reward + 10000
            #print('Added: ' + str(reward))
        else:
            # put a penalty if the no of time steps is more
            reward = reward - 10  
            #print('Deducted: ' + str(reward))

        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        scores += reward
        if done:
            flag = 1
            agent.update_target_model()
            print("episode: {}/{}, score: {}, e: {:.2}"
                  .format(e, EPISODES, scores, agent.epsilon))
                
            break

        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
            
    if flag == 0:
        print("episode: {}/{}, score: {}, e: {:.2}".format(e, EPISODES, time, agent.epsilon))      
        
    if round(agent.epsilon,2) == 0.01:
        print("The top has been reached")
        break 
        
    if e % 100 == 0:
        print('saving the model')
        agent.save("mountain_car-dqn.h5")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
state size: 2
action size:  3
episode: 0/400, score: 9, e: 1.0
saving the model
episode: 1/400, score: 9, e: 1.0
episode: 2/400, score: 9, e: 1.0
episode: 3/400, score: 9, e: 1.0
episode: 4/400, score: 9, e: 1.0
episode: 5/400, score: 9, e: 1.0
episode: 6/400, score: 9, e: 1.0
episode: 7/400, score: 9, e: 1.0
episode: 8/400, score: 9, e: 1.0
episode: 9/400, score: 9, e: 1.0
episode: 10/400, score: 9, e: 1.0
episode: 11/400, score: 9, e: 1.0
episode: 12/400, score: 9, e: 0.99
episode: 13/400, score: 9, e: 0.96
episode: 14/400, score: 9, e: 0.94
episode: 15/400, score: 9, e: 0.91
episode: 16/400, score: 9, e: 0.88
episode: 17/400, score: 9, e: 0.86
episode: 18/400, score: 9, e: 0.83
episode: 19/400, score: 9, e: 0.81
episode: 20/400, score: 9, e: 0.78
episode: 21/400, score: 9, e: 0.76
episode: 22/400, score: 9, e: 0.74
episode: 23/400, score: 9, e: 0.71
episode: 24/400, score: 9, e: 0.69
e

## Visualisation

In [9]:
HTML('<img src="Mountain Car.gif">')